In [1]:
import numpy as np
from simple.ohlc import renko
from simple.plotly import chartFigure, chartTrades
from simple.funcs import symlog
from simple.backtest import npBacktestMarket

In [2]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)[:100_000]
len(T)

100000

In [3]:
Buy = T[T.Size > 0]
Sell = T[T.Size < 0]
Buy[:5]

rec.array([('2022-05-08T00:01:00.335000', 2735, 2636.,  6.),
           ('2022-05-08T00:01:00.335000', 2736, 2636.,  4.),
           ('2022-05-08T00:01:00.668000', 2737, 2636., 96.),
           ('2022-05-08T00:01:00.668000', 2738, 2636.,  2.),
           ('2022-05-08T00:01:00.668000', 2739, 2636.,  2.)],
          dtype={'names':['DT','LocalDT','Price','Size'], 'formats':['<M8[us]','<i8','<f8','<f8'], 'offsets':[0,8,16,24], 'itemsize':40})

In [4]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S[:5]

rec.array([('2022-05-08T00:01:00.007000', 2636., 2635.99),
           ('2022-05-08T00:01:00.043000', 2636., 2635.99),
           ('2022-05-08T00:01:00.079000', 2636., 2635.99),
           ('2022-05-08T00:01:00.117000', 2636., 2635.99),
           ('2022-05-08T00:01:00.149000', 2636., 2635.99)],
          dtype={'names':['DT','A','B'], 'formats':['<M8[us]','<f4','<f4'], 'offsets':[0,8,16], 'itemsize':24})

In [5]:
len(T), len(S)

(100000, 600964)

In [6]:
C = renko(T, 15)
C[:5], len(C)

[(0, 17610, 2625.0, 2640.0),
 (17611, 18413, 2655.0, 2640.0),
 (18414, 18420, 2625.0, 2640.0),
 (18421, 18421, 2655.0, 2640.0),
 (18422, 52272, 2640.0, 2625.0)]

In [9]:
s = 0
Signal = np.zeros(len(S.DT))

for c in C:
    color = 'green' if c[3] > c[2] else 'red'
    duration = T.DT[c[1]] - T.DT[c[0]]
    tm = T.DT[c[1]] + delay
    k = np.searchsorted(S.DT, tm)
    if duration < np.timedelta64(10000, 'μs'):
        if c[3] > c[2]:
            s = 1
        elif c[3] < c[2]:
            s = -1
    if k < len(Signal):
        Signal[k] = s

    print(f'{str(c): <26}', T.DT[c[1]], tm, f'{duration: <16}', k, s, color, sep='\t')

(0, 17610, 2625.0, 2640.0)	2022-05-08T00:10:12.393999	2022-05-08T00:10:12.493999	552058999 microseconds	15754	0	green
(17611, 18413, 2655.0, 2640.0)	2022-05-08T00:10:36.481000	2022-05-08T00:10:36.581000	24072001 microseconds	16437	0	red
(18414, 18420, 2625.0, 2640.0)	2022-05-08T00:10:36.481000	2022-05-08T00:10:36.581000	0 microseconds  	16437	1	green
(18421, 18421, 2655.0, 2640.0)	2022-05-08T00:10:36.496000	2022-05-08T00:10:36.596000	0 microseconds  	16438	-1	red
(18422, 52272, 2640.0, 2625.0)	2022-05-08T00:30:48.915000	2022-05-08T00:30:49.015000	1212399000 microseconds	50471	-1	red
(52273, 52353, 2610.0, 2625.0)	2022-05-08T00:30:48.921000	2022-05-08T00:30:49.021000	0 microseconds  	50471	1	green
(52354, 52376, 2640.0, 2625.0)	2022-05-08T00:30:49.108999	2022-05-08T00:30:49.208999	175999 microseconds	50477	1	red
(52377, 52381, 2610.0, 2625.0)	2022-05-08T00:30:49.116999	2022-05-08T00:30:49.216999	8000 microseconds	50477	1	green
(52382, 52382, 2640.0, 2625.0)	2022-05-08T00:30:49.135999	20

In [10]:
trades = npBacktestMarket(S.DT.astype(np.int64), S.A, S.B, Signal, 0)
trades

rec.array([(16438, '2022-05-08T00:10:36.622000', 2639.86010742, 2639.83007812, 16439, '2022-05-08T00:10:36.658000', 2639.80004883, 2639.83007812,  1.),
           (50472, '2022-05-08T00:30:49.057999', 2625.01000977, 2625.        , 50479, '2022-05-08T00:30:49.285999', 2624.35009766, 2624.45019531,  1.),
           (50823, '2022-05-08T00:31:00.953999', 2625.33007812, 2625.22998047, 55718, '2022-05-08T00:33:54.014000', 2624.82006836, 2624.84008789,  1.),
           (56188, '2022-05-08T00:34:10.736000', 2625.05004883, 2625.05517578, 82350, '2022-05-08T00:49:40.028000', 2624.92993164, 2624.81494141, -1.),
           (82351, '2022-05-08T00:49:40.060000', 2624.89990234, 2624.91503906, 83420, '2022-05-08T00:50:17.009999', 2625.01000977, 2625.00488281, -1.)],
          dtype=[('X0', '<i8'), ('T0', '<M8[us]'), ('Price0', '<f8'), ('MidPrice0', '<f8'), ('X1', '<i8'), ('T1', '<M8[us]'), ('Price1', '<f8'), ('MidPrice1', '<f8'), ('Size', '<f8')])

In [11]:
# timed chart
fig = chartFigure(rows=2,
    Sell=dict(x=Sell.DT, y=Sell.Price, color='red', mode='markers', size=-symlog(Sell.Size), opacity=0.2),
    Buy=dict(x=Buy.DT, y=Buy.Price, color='green', mode='markers', size=symlog(Buy.Size), opacity=0.2),
    Ask=dict(x=S.DT, y=S.A, color='red', opacity=0.2, shape='hv', mode='lines'),
    Bid=dict(x=S.DT, y=S.B, color='green', opacity=0.2, shape='hv', mode='lines'),
    Signal=dict(x=S.DT, y=Signal, row=2, color='magenta', shape='hv'),
    **chartTrades(trades)
)
fig

FigureWidgetResampler({
    'data': [{'marker': {'color': 'red',
                         'opacity': 0.2,
    …

In [12]:
fig.layout.shapes = [{
    'fillcolor': 'green' if c[3] > c[2] else 'red',
    'line': {'color': 'black', 'width': 2},
    'opacity': 0.15,
    'type': 'rect',
    'x0': str(T.DT[int(c[0])]),
    'x1': str(T.DT[int(c[1])]),
    'y0': c[2],
    'y1': c[3]
} for c in C]